# 医疗文档图像文字识别演示

> 🏥 使用PaddleOCR从医疗文档PNG图像中提取文本并保存为CSV

**版本**: v1.1.1 (修复API兼容性) | **更新时间**: 2025-08-17

## 🎯 功能特性
- 📄 支持医疗文档图像文字识别
- 🤖 使用PaddleOCR高精度识别引擎
- 📊 自动生成结构化CSV报告
- 🖼️ 支持多种图像格式输入
- 💡 简单易用的交互界面

## 🚀 使用说明
1. 运行环境检查和依赖安装
2. 上传医疗文档图像
3. 执行OCR文字识别
4. 下载CSV结果文件

---
*使用 Claude Code 开发，Google Colab 运行 🚀*

In [ ]:
# ================================
# 环境检查和基础设置
# ================================

import warnings
warnings.filterwarnings("ignore")

def check_environment():
    """检查运行环境并显示系统信息"""
    print("🔍 检查运行环境...")
    
    # 检查是否在Colab环境
    try:
        import google.colab  # noqa: F401
        print("✅ 运行在Google Colab")
        in_colab = True
    except ImportError:
        print("ℹ️ 运行在本地环境")
        in_colab = False
    
    # 检查GPU
    try:
        import torch
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"✅ 计算设备: {device}")
        if device == 'cuda':
            print(f"✅ GPU型号: {torch.cuda.get_device_name(0)}")
            print(f"✅ GPU内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    except ImportError:
        print("ℹ️ PyTorch未安装，使用CPU模式")
    
    return in_colab

# 运行环境检查
in_colab = check_environment()

In [ ]:
# ================================
# 安装必要的依赖包
# ================================

def install_dependencies():
    """安装项目所需的依赖包"""
    print("📦 安装医疗OCR项目依赖...")
    
    import subprocess
    import sys
    
    # 核心依赖包列表
    packages = [
        'paddlepaddle',
        'paddleocr',
        'pandas',
        'pillow',
        'opencv-python',
        'tqdm',
        'gradio'
    ]
    
    for package in packages:
        try:
            if package == 'opencv-python':
                import cv2  # noqa: F401
                print(f"✅ {package} 已安装")
            elif package == 'pillow':
                from PIL import Image  # noqa: F401
                print(f"✅ {package} 已安装")
            else:
                __import__(package.replace('-', '_'))
                print(f"✅ {package} 已安装")
        except ImportError:
            print(f"📥 安装 {package}...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
    
    print("✅ 所有依赖安装完成!")

# 安装依赖
install_dependencies()

In [ ]:
# ================================
# 导入必要的库
# ================================

try:
    import pandas as pd
    from PIL import Image
    from tqdm import tqdm
    from paddleocr import PaddleOCR
    import gradio as gr
    print("📚 所有库导入成功!")
except ImportError as e:
    print(f"❌ 库导入失败: {e}")
    print("💡 请先运行依赖安装单元格")

In [ ]:
# ================================
# 医疗OCR核心功能类
# ================================

class MedicalOCRProcessor:
    def __init__(self):
        """初始化医疗OCR处理器"""
        print("🏥 初始化医疗OCR处理器...")
        
        # 检查GPU可用性
        try:
            import torch
            use_gpu = torch.cuda.is_available()
            gpu_info = f"GPU可用: {use_gpu}"
            if use_gpu:
                gpu_info += f" (设备: {torch.cuda.get_device_name(0)})"
            print(f"⚡ {gpu_info}")
        except ImportError:
            use_gpu = False
            print("ℹ️ PyTorch未安装，使用CPU模式")
        
        # 初始化PaddleOCR，支持中英文
        # 新版PaddleOCR会自动检测并使用GPU（如果可用）
        self.ocr = PaddleOCR(use_angle_cls=True, lang='ch')
        
        print("✅ OCR引擎初始化完成")
    
    def extract_text_from_image(self, image_path):
        """从图像中提取文字"""
        try:
            # 使用PaddleOCR识别文字（移除cls参数以兼容新版API）
            result = self.ocr.ocr(image_path)
            
            # 提取文字内容
            extracted_texts = []
            if result and result[0]:
                for line in result[0]:
                    text = line[1][0]
                    confidence = line[1][1]
                    extracted_texts.append({
                        'text': text,
                        'confidence': confidence
                    })
            
            return extracted_texts
        
        except Exception as e:
            print(f"❌ 图像处理失败: {str(e)}")
            return []
    
    def process_single_image(self, image_path):
        """处理单个图像文件"""
        import os
        print(f"📄 处理图像: {os.path.basename(image_path)}")
        
        # 提取文字
        extracted_texts = self.extract_text_from_image(image_path)
        
        # 整理结果
        results = []
        for i, item in enumerate(extracted_texts):
            results.append({
                'file_name': os.path.basename(image_path),
                'line_number': i + 1,
                'extracted_text': item['text'],
                'confidence': round(item['confidence'], 4)
            })
        
        return results
    
    def process_multiple_images(self, image_paths):
        """批量处理多个图像文件"""
        all_results = []
        
        print(f"📊 开始批量处理 {len(image_paths)} 个图像文件...")
        
        for image_path in tqdm(image_paths, desc="处理进度"):
            results = self.process_single_image(image_path)
            all_results.extend(results)
        
        return all_results
    
    def save_results_to_csv(self, results, output_path):
        """保存结果到CSV文件"""
        df = pd.DataFrame(results)
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"💾 结果已保存到: {output_path}")
        return df

# 初始化OCR处理器
ocr_processor = MedicalOCRProcessor()

In [ ]:
# ================================
# 创建示例医疗文档（用于演示）- 修复中文字体
# ================================

def create_sample_medical_document():
    """创建示例医疗文档图像用于演示"""
    from PIL import Image, ImageDraw, ImageFont
    import requests
    import os
    import subprocess
    
    # 创建示例图像
    img = Image.new('RGB', (800, 600), color='white')
    draw = ImageDraw.Draw(img)
    
    # 添加示例医疗文档内容
    sample_text = [
        "医院名称：XX市人民医院",
        "患者姓名：张三", 
        "性别：男    年龄：45岁",
        "科室：心血管内科",
        "主治医师：李医生",
        "诊断：高血压、糖尿病",
        "处方：",
        "1. 降压药 10mg 每日一次",
        "2. 降糖药 5mg 每日两次",
        "医生签名：李医生",
        "日期：2024-08-17"
    ]
    
    # 安装并获取中文字体
    font = None
    font_size = 24
    
    try:
        # 方案1: 安装系统中文字体包
        print("📥 安装中文字体支持...")
        try:
            subprocess.run(['apt-get', 'update'], check=False, capture_output=True)
            subprocess.run(['apt-get', 'install', '-y', 'fonts-noto-cjk'], 
                         check=False, capture_output=True)
            print("✅ 中文字体包安装完成")
        except:
            pass
        
        # 方案2: 搜索可用的中文字体
        chinese_fonts = [
            # Noto字体 (Google)
            '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc',
            '/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc',
            '/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf',
            '/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf',
            # 其他可能的字体路径
            '/System/Library/Fonts/Arial.ttf',  # Mac
            'C:/Windows/Fonts/simhei.ttf',      # Windows
        ]
        
        for font_path in chinese_fonts:
            if os.path.exists(font_path):
                try:
                    font = ImageFont.truetype(font_path, font_size)
                    print(f"✅ 使用字体: {font_path}")
                    break
                except Exception as e:
                    print(f"⚠️ 字体加载失败: {font_path} - {e}")
                    continue
        
        # 方案3: 下载可靠的中文字体
        if font is None:
            print("📥 下载NotoSans中文字体...")
            try:
                # 使用可靠的字体URL
                font_urls = [
                    "https://github.com/notofonts/noto-cjk/raw/main/Sans/OTF/SimplifiedChinese/NotoSansSC-Regular.otf",
                    "https://fonts.gstatic.com/s/notosanssc/v31/HI_SiYsKILxRpg3hIP6sJ7fM7PqlM-vWjMY.otf"
                ]
                
                for font_url in font_urls:
                    try:
                        font_response = requests.get(font_url, timeout=15)
                        if font_response.status_code == 200:
                            font_path = "NotoSansSC-Regular.otf"
                            with open(font_path, 'wb') as f:
                                f.write(font_response.content)
                            font = ImageFont.truetype(font_path, font_size)
                            print(f"✅ 中文字体下载成功: {font_url}")
                            break
                    except Exception as e:
                        print(f"⚠️ 字体下载失败: {font_url} - {e}")
                        continue
                        
                if font is None:
                    raise Exception("所有字体下载方案都失败")
                    
            except Exception as e:
                print(f"⚠️ 字体下载失败: {e}")
                
    except Exception as e:
        print(f"⚠️ 中文字体获取失败: {e}")
    
    # 方案4: 使用默认字体作为备用
    if font is None:
        try:
            # 使用PIL默认字体
            font = ImageFont.load_default()
            print("✅ 使用默认字体")
        except:
            print("⚠️ 默认字体也无法加载")
            font = None
    
    # 绘制文本
    y_position = 50
    line_height = 40
    
    for text in enumerate(sample_text):
        try:
            if font:
                # 尝试绘制中文文本
                draw.text((50, y_position), text[1], fill='black', font=font)
                print(f"✅ 绘制文本: {text[1][:10]}...")
            else:
                # 无字体时使用默认绘制
                draw.text((50, y_position), text[1], fill='black')
                print(f"⚠️ 无字体绘制: {text[1][:10]}...")
                
        except Exception as e:
            # 如果中文字体失败，转换为英文描述
            print(f"⚠️ 文字绘制失败: {e}")
            try:
                english_text = translate_to_english(text[1])
                if font:
                    draw.text((50, y_position), english_text, fill='black', font=font)
                else:
                    draw.text((50, y_position), english_text, fill='black')
                print(f"✅ 使用英文备用: {english_text[:20]}...")
            except Exception as e2:
                print(f"❌ 英文绘制也失败: {e2}")
        
        y_position += line_height
    
    # 保存示例图像
    sample_path = 'sample_medical_document.png'
    img.save(sample_path)
    print(f"📄 创建示例医疗文档: {sample_path}")
    
    return sample_path

def translate_to_english(chinese_text):
    """将中文文本转换为英文（备用方案）"""
    translations = {
        "医院名称：XX市人民医院": "Hospital: XX City People's Hospital",
        "患者姓名：张三": "Patient Name: Zhang San",
        "性别：男    年龄：45岁": "Gender: Male    Age: 45",
        "科室：心血管内科": "Department: Cardiology",
        "主治医师：李医生": "Doctor: Dr. Li",
        "诊断：高血压、糖尿病": "Diagnosis: Hypertension, Diabetes",
        "处方：": "Prescription:",
        "1. 降压药 10mg 每日一次": "1. Antihypertensive 10mg once daily",
        "2. 降糖药 5mg 每日两次": "2. Antidiabetic 5mg twice daily",
        "医生签名：李医生": "Doctor Signature: Dr. Li",
        "日期：2024-08-17": "Date: 2024-08-17"
    }
    return translations.get(chinese_text, chinese_text)

# 创建示例文档
print("🎨 创建示例医疗文档...")
sample_doc = create_sample_medical_document()

# 显示示例图像
try:
    from IPython.display import Image as IPImage, display  # noqa: F401
    display(IPImage(sample_doc))
except ImportError:
    # 在非Jupyter环境中的备用方案
    print(f"✅ 示例文档已创建: {sample_doc}")
    print("💡 在Jupyter/Colab环境中会自动显示图像")

In [ ]:
# ================================
# 演示OCR功能
# ================================

def demo_ocr_functionality():
    """演示OCR功能"""
    print("🚀 开始演示医疗OCR功能...")
    
    # 处理示例文档
    results = ocr_processor.process_single_image(sample_doc)
    
    # 显示识别结果
    print("\n📊 文字识别结果:")
    print("-" * 60)
    
    for result in results:
        print(f"行{result['line_number']:2d}: {result['extracted_text']} (置信度: {result['confidence']:.3f})")
    
    # 保存结果到CSV
    csv_path = 'ocr_results_demo.csv'
    df = ocr_processor.save_results_to_csv(results, csv_path)
    
    print(f"\n📈 共识别出 {len(results)} 行文字")
    print(f"📄 结果已保存到 CSV 文件: {csv_path}")
    
    # 显示CSV内容预览
    print("\n📋 CSV文件预览:")
    print(df.to_string(index=False))
    
    return df, csv_path

# 运行演示
demo_df, demo_csv = demo_ocr_functionality()

In [ ]:
# ================================
# 创建交互式界面
# ================================

def create_gradio_interface():
    """创建Gradio交互界面"""
    
    def process_uploaded_image(image):
        """处理上传的图像"""
        if image is None:
            return "请上传图像文件", None
        
        try:
            # 保存临时文件
            temp_path = "temp_uploaded_image.png"
            Image.fromarray(image).save(temp_path)
            
            # 处理图像
            results = ocr_processor.process_single_image(temp_path)
            
            if not results:
                return "未识别到文字内容", None
            
            # 生成结果文本
            result_text = "📊 识别结果:\n\n"
            for i, result in enumerate(results, 1):
                result_text += f"{i:2d}. {result['extracted_text']} (置信度: {result['confidence']:.3f})\n"
            
            # 保存CSV文件
            csv_path = "ocr_results.csv"
            ocr_processor.save_results_to_csv(results, csv_path)
            
            result_text += f"\n✅ 共识别 {len(results)} 行文字"
            result_text += f"\n💾 结果已保存到: {csv_path}"
            
            return result_text, csv_path
            
        except Exception as e:
            return f"❌ 处理失败: {str(e)}", None
    
    # 创建界面
    interface = gr.Interface(
        fn=process_uploaded_image,
        inputs=[
            gr.Image(label="上传医疗文档图像", type="numpy")
        ],
        outputs=[
            gr.Textbox(label="识别结果", lines=15),
            gr.File(label="下载CSV结果文件")
        ],
        title="🏥 医疗文档OCR识别系统",
        description="上传医疗文档图像，自动识别其中的文字内容并生成CSV报告。支持中英文混合识别。",
        examples=[
            [sample_doc]
        ],
        theme=gr.themes.Soft()
    )
    
    return interface

# 创建并启动界面
print("🌐 创建交互界面...")
interface = create_gradio_interface()

if interface:
    print("✅ 界面创建成功!")
    print("🚀 启动界面服务...")
    interface.launch(
        share=True,
        debug=True,
        show_error=True
    )
else:
    print("❌ 界面创建失败")

In [ ]:
# ================================
# 项目总结和使用指南
# ================================

def show_project_summary():
    """显示项目总结和使用指南"""
    
    summary = """
🎉 医疗文档OCR识别演示项目运行完成!

📋 项目功能总结:
✅ 医疗文档图像文字识别
✅ 中英文混合文字识别
✅ 批量图像处理支持
✅ CSV格式结果导出
✅ 置信度评估
✅ 用户友好的交互界面

🛠️ 技术特性:
• 基于PaddleOCR高精度识别引擎
• 支持GPU加速处理
• 自动角度检测和矫正
• 结构化数据输出

📖 使用场景:
• 医疗处方识别
• 病历文档数字化
• 检查报告提取
• 医疗档案管理

🔗 项目地址:
GitHub: https://github.com/zhurong2020/mcr
Colab: 当前notebook链接

👨‍⚕️ 适用用户:
• 医疗工作者
• 数据分析师
• 研究人员
• 开发者

⚠️ 使用说明:
1. 确保图像清晰可读
2. 支持PNG、JPG等常见格式
3. 批量处理时注意文件大小
4. 结果仅供参考，重要信息请人工核验

💡 技术支持:
如有问题或建议，欢迎反馈交流！
    """
    
    print(summary)

# 显示项目总结
show_project_summary()